In [292]:
import re
import string

def clean_msg(msg):
    
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg



import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
import nltk
p_stemmer = PorterStemmer()

def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
  
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

import xlrd  
import pandas as pd
Corpus = pd.read_excel('corpus_comments.xlsx')
Corpus['comments'] = Corpus['comments'].astype('str')
Corpus['reaction_num'] = Corpus['reaction_num'].astype('float64')
Corpus['reply_num'] = Corpus['reply_num'].astype('float64')
Corpus['reaction1'] = Corpus['reaction1'].astype('str')
Corpus['reaction2'] = Corpus['reaction2'].astype('str')
Corpus['reaction3'] = Corpus['reaction3'].astype('str')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Train_X, Test_X, Train_Y, Test_Y = train_test_split(Corpus[['comments','reaction_num','reply_num','reaction1','reaction2','reaction3']], Corpus[['Rude','Figurative','Offensive','Dirty']], test_size=0.2,random_state=1000);

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
#vectorizer = CountVectorizer()
#vectorizer = CountVectorizer(tokenizer = split_word)
#vectorizer.fit(Train_X)
#vectorizer.vocabulary_

#BOW
#X_train = vectorizer.transform(Train_X).toarray()
#X_test  = vectorizer.transform(Test_X).toarray()

#Tokenizaiton
Tfidf_vect = TfidfVectorizer(tokenizer = split_word)
#Dictionary
Tfidf_vect.fit(Corpus['comments'])
#print(Tfidf_vect.vocabulary_)
#Tf-idf
Train_X_Tfidf = Tfidf_vect.transform(Train_X['comments'])
Test_X_Tfidf = Tfidf_vect.transform(Test_X['comments'])




In [293]:
from sklearn.metrics import classification_report
from  sklearn.metrics import precision_recall_fscore_support
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    
#     total = support.sum() 
#     avg[-1] = total

#     class_report_df['avg / total'] = avg

    return class_report_df.T

In [294]:
import csv
import itertools
def writeEvaluation(s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16):
    f = open("classification_report.csv", mode='a')
    f.write(s1+","+s2+","+s3+","+s4)
    f.write("\n")
    f.write(s5+","+s6+","+s7+","+s8)
    f.write("\n")
    f.write(s9+","+s10+","+s11+","+s12)
    f.write("\n")
    f.write(s13+","+s14+","+s15+","+s16)
    f.write("\n")
    f.close()
    
def writeEvaluation_threshold_1_minus_threshold(s1,s2,s3,s4,s5,s6,s7,s8):
    f = open("classification_report.csv", mode='a')
    f.write(s1+","+s2+","+s3+","+s4)
    f.write("\n")
    f.write(s5+","+s6+","+s7+","+s8)
    f.write("\n")
    f.close()

In [295]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
string='';
def report_performance(predictions_class_train, predictions_class_test, predictions_class_train_c, predictions_class_test_c, type_class,threshold):
    #Evaluation class (test data) 
    str1 = "Test data (ensemble)"
    str2 = str(threshold)
    str3 = str(type_class)
    str4 = str(accuracy_score(predictions_class_test, Test_Y[type_class]))
    print(type_class+" Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_test, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test))
    report = classification_report(Test_Y[type_class],predictions_class_test)
    print("PRF: \n", report)
    
    print("-----------------------------------------------------------\n")
    str5 = "Test data (multiple features)"
    str6 = str(threshold)
    str7 = str(type_class)
    str8 = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score(test data with multiple features):", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    report = classification_report(Test_Y[type_class],predictions_class_test_c);
    print("PRF: \n", report)

    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    str9 = "Train data (ensemble)"
    str10 = str(threshold)
    str11 = str(type_class)
    str12 = str(accuracy_score(predictions_class_train, Train_Y[type_class]))
    print(type_class+" Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_train, Train_Y[type_class])*100)
    print("Confusion Matrix: \n",confusion_matrix(Train_Y[type_class],predictions_class_train))
    print("PRF: \n",classification_report(Train_Y[type_class],predictions_class_train))
    print("-----------------------------------------------------------\n")
    str13 = "Train data (mulitple features)"
    str14 = str(threshold)
    str15 = str(type_class)
    str16 = str(accuracy_score(predictions_class_train_c, Train_Y[type_class]))
    print(type_class+" Accuracy score(train data with multiple features):", 
          accuracy_score(predictions_class_train_c, Train_Y[type_class])*100)
    print("Confusion Matrix: \n",confusion_matrix(Train_Y[type_class],predictions_class_train_c))
    print("PRF: \n",classification_report(Train_Y[type_class],predictions_class_train_c))
    print("___________________________________________________________________________________________________________\n")

    writeEvaluation(str1,str2,str3,str4,str5,str6,str7,str8,str9,str10,str11,str12,str13,str14,str15,str16)


In [296]:
string='';
def report_performance_threshold_1_minus_threshold(predictions_class_train, predictions_class_test, type_class,threshold):
     #Evaluation class (test data) 
    str1 = "Test data ensemble(content context (threshold & 1-threshold))"
    str2 = str(threshold)
    str3 = str(type_class)
    str4 = str(accuracy_score(predictions_class_test, Test_Y[type_class]))
    print(type_class+" Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_test, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test))
    report = classification_report(Test_Y[type_class],predictions_class_test)
    print("PRF: \n", report)
    print("-----------------------------------------------------------\n")
    
    #Evaluation class (train data)
    str5 = "Train data ensemble(content context (threshold & 1-threshold)"
    str6 = str(threshold)
    str7 = str(type_class)
    str8 = str(accuracy_score(predictions_class_train, Train_Y[type_class]))
    print(type_class+" Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature): ",
          accuracy_score(predictions_class_train, Train_Y[type_class])*100)
    print("Confusion Matrix: \n",confusion_matrix(Train_Y[type_class],predictions_class_train))
    print("PRF: \n",classification_report(Train_Y[type_class],predictions_class_train))
    print("___________________________________________________________________________________________________________\n")

    writeEvaluation_threshold_1_minus_threshold(str1,str2,str3,str4,str5,str6,str7,str8)


In [297]:
def predict_class(list_predict_prob,threshold):
    list_predict_class = [];
    for i in range (len(list_predict_prob)):
        if(list_predict_prob[i][1] > threshold):
            list_predict_class.append(1)
        else :
            list_predict_class.append(0)
    return list_predict_class

In [298]:
def changeStrInReactionToNum(reaction_data=[]):
    reaction = []
    for i in reaction_data:
        if(i == 'haha'):
            reaction.append(1)
        elif(i == 'like'):
            reaction.append(2)
        elif(i == 'angry'):
            reaction.append(3)
        elif(i == 'sad'):
            reaction.append(4)
        elif(i == 'wow'):
            reaction.append(5)
        elif(i == 'love'):
            reaction.append(6)
        else:
            reaction.append(7)
    return reaction

In [299]:
Train_X_Tfidf

<2155x5498 sparse matrix of type '<class 'numpy.float64'>'
	with 28085 stored elements in Compressed Sparse Row format>

In [300]:
from scipy.sparse import csr_matrix
train_reaction_num = csr_matrix(Train_X['reaction_num'].values.reshape(-1, 1))
test_reaction_num = csr_matrix(Test_X['reaction_num'].values.reshape(-1, 1))
train_reply_num = csr_matrix(Train_X['reply_num'].values.reshape(-1,1))
test_reply_num = csr_matrix(Test_X['reply_num'].values.reshape(-1,1))

In [301]:
import numpy as np
# Train_X['reaction1'] = np.nan_to_num(0)
# Test_X['reaction1'] = np.nan_to_num(0)
# Train_X['reaction2'] = np.nan_to_num(0)
# Test_X['reaction2'] = np.nan_to_num(0)
# Train_X['reaction3'] = np.nan_to_num(0)
# Test_X['reaction3'] = np.nan_to_num(0)
# train_reaction1_feat = csr_matrix(Train_X['reaction1'].values.reshape(-1, 1))
# test_reaction1_feat = csr_matrix(Test_X['reaction1'].values.reshape(-1, 1))
# train_reaction2_feat = csr_matrix(Train_X['reaction2'].values.reshape(-1, 1))
# test_reaction2_feat = csr_matrix(Test_X['reaction2'].values.reshape(-1, 1))
# train_reaction3_feat = csr_matrix(Train_X['reaction3'].values.reshape(-1, 1))
# test_reaction3_feat = csr_matrix(Test_X['reaction3'].values.reshape(-1, 1))
train_reaction1 = changeStrInReactionToNum(Train_X['reaction1'])
test_reaction1 = changeStrInReactionToNum(Test_X['reaction1'])
train_reaction2 = changeStrInReactionToNum(Train_X['reaction2'])
test_reaction2 =  changeStrInReactionToNum(Test_X['reaction2'])
train_reaction3 = changeStrInReactionToNum(Train_X['reaction3'])
test_reaction3=  changeStrInReactionToNum(Test_X['reaction3'])

train_reaction1_feat = csr_matrix(train_reaction1).reshape(-1,1)
test_reaction1_feat = csr_matrix(test_reaction1).reshape(-1,1)
train_reaction2_feat = csr_matrix(train_reaction2).reshape(-1,1)
test_reaction2_feat = csr_matrix(test_reaction2).reshape(-1,1)
train_reaction3_feat = csr_matrix(train_reaction3).reshape(-1,1)
test_reaction3_feat = csr_matrix(test_reaction3).reshape(-1,1)

In [302]:
from scipy.sparse import hstack, csr_matrix
train_feat = hstack([Train_X_Tfidf, train_reaction_num, train_reply_num, train_reaction1_feat, train_reaction2_feat, train_reaction3_feat])
test_feat = hstack([Test_X_Tfidf, test_reaction_num, test_reply_num, test_reaction1_feat, test_reaction2_feat, test_reaction3_feat])
train_feat_context = hstack([train_reaction_num, train_reply_num, train_reaction1_feat, train_reaction2_feat, train_reaction3_feat])
test_feat_context = hstack([test_reaction_num, test_reply_num, test_reaction1_feat, test_reaction2_feat, test_reaction3_feat])

In [303]:
#Naive bays
import numpy
from sklearn.naive_bayes import MultinomialNB
#ensemble model of comments feature and context feature
abusive_type = ["Rude","Figurative","Offensive","Dirty"]
report=[]
header = 0;
for threshold in numpy.arange(0.1,1.1,0.1):
    for i in range(0,4):
        Naive_bayes_1 = MultinomialNB()
        Naive_bayes_2 = MultinomialNB()

        Naive_bayes_1.fit(Train_X_Tfidf, Train_Y[abusive_type[i]])
        Naive_bayes_2.fit(train_feat_context, Train_Y[abusive_type[i]])

        predictions_NB_Train1 = Naive_bayes_1.predict_proba(Train_X_Tfidf)
        predictions_NB_Train2 = Naive_bayes_2.predict_proba(train_feat_context)

        predictions_NB_Test1 = Naive_bayes_1.predict_proba(Test_X_Tfidf)
        predictions_NB_Test2 = Naive_bayes_2.predict_proba(test_feat_context)

        predictions_NB_Train = (predictions_NB_Train1+predictions_NB_Train2)/2
        predictions_NB_Test = (predictions_NB_Test1+predictions_NB_Test2)/2

        predictions_class_Train = predict_class(predictions_NB_Train,threshold)
        predictions_class_Test = predict_class(predictions_NB_Test,threshold)

        #combine multiple feature with one model
        Naive_bayes_3 = MultinomialNB()
        Naive_bayes_3.fit(train_feat, Train_Y[abusive_type[i]])
        predictions_NB_Train3 = Naive_bayes_3.predict_proba(train_feat)
        predictions_NB_Test3 = Naive_bayes_3.predict_proba(test_feat)

        predictions_class_Train_c = (predictions_NB_Train3 [:,1] >= threshold).astype('int')
        predictions_class_Test_c = (predictions_NB_Test3 [:,1] >= threshold).astype('int')
    #     predictions_class_Train_c = Naive_bayes_3.predict(train_feat)
    #     predictions_class_Test_c = Naive_bayes_3.predict(test_feat)
        report_performance(predictions_class_Train, predictions_class_Test, predictions_class_Train_c,predictions_class_Test_c, abusive_type[i],threshold)

Rude Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  63.4508348794063
Confusion Matrix: 
 [[  6 197]
 [  0 336]]
PRF: 
               precision    recall  f1-score   support

           0       1.00      0.03      0.06       203
           1       0.63      1.00      0.77       336

    accuracy                           0.63       539
   macro avg       0.82      0.51      0.42       539
weighted avg       0.77      0.63      0.50       539

-----------------------------------------------------------

Rude Accuracy score(test data with multiple features): 64.56400742115028
Confusion Matrix: 
 [[ 27 176]
 [ 15 321]]
PRF: 
               precision    recall  f1-score   support

           0       0.64      0.13      0.22       203
           1       0.65      0.96      0.77       336

    accuracy                           0.65       539
   macro avg       0.64      0.54      0.50       539
weighted avg       0.64      0.65      0.56       5

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

Dirty Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  91.4656771799629
Confusion Matrix: 
 [[493  43]
 [  3   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      0.92      0.96       536
           1       0.00      0.00      0.00         3

    accuracy                           0.91       539
   macro avg       0.50      0.46      0.48       539
weighted avg       0.99      0.91      0.95       539

-----------------------------------------------------------

Dirty Accuracy score(test data with multiple features): 99.44341372912801
Confusion Matrix: 
 [[536   0]
 [  3   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       536
           1       0.00      0.00      0.00         3

    accuracy                           0.99       539
   macro avg       0.50      0.50      0.50       539
weighted avg       0.99      0.99      0.99      

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

Figurative Accuracy score(train data with multiple features): 95.22041763341068
Confusion Matrix: 
 [[2052    0]
 [ 103    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      2052
           1       0.00      0.00      0.00       103

    accuracy                           0.95      2155
   macro avg       0.48      0.50      0.49      2155
weighted avg       0.91      0.95      0.93      2155

___________________________________________________________________________________________________________

Offensive Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  39.332096474953616
Confusion Matrix: 
 [[191 317]
 [ 10  21]]
PRF: 
               precision    recall  f1-score   support

           0       0.95      0.38      0.54       508
           1       0.06      0.68      0.11        31

    accuracy                           0.39       539
   macro avg       0.51      0.53      

...

Confusion Matrix: 
 [[ 328  537]
 [ 124 1166]]
PRF: 
               precision    recall  f1-score   support

           0       0.73      0.38      0.50       865
           1       0.68      0.90      0.78      1290

    accuracy                           0.69      2155
   macro avg       0.71      0.64      0.64      2155
weighted avg       0.70      0.69      0.67      2155

-----------------------------------------------------------

Rude Accuracy score(train data with multiple features): 67.05336426914154
Confusion Matrix: 
 [[ 256  609]
 [ 101 1189]]
PRF: 
               precision    recall  f1-score   support

           0       0.72      0.30      0.42       865
           1       0.66      0.92      0.77      1290

    accuracy                           0.67      2155
   macro avg       0.69      0.61      0.59      2155
weighted avg       0.68      0.67      0.63      2155

_______________________________________________________________________________________________________

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

Dirty Accuracy score(test data with multiple features): 99.44341372912801
Confusion Matrix: 
 [[536   0]
 [  3   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       536
           1       0.00      0.00      0.00         3

    accuracy                           0.99       539
   macro avg       0.50      0.50      0.50       539
weighted avg       0.99      0.99      0.99       539

-----------------------------------------------------------

Dirty Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  98.65429234338747
Confusion Matrix: 
 [[2126    0]
 [  29    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      2126
           1       0.00      0.00      0.00        29

    accuracy                           0.99      2155
   macro avg       0.49      0.50      0.50      2155
weighted avg       0.97      0.99      0.98

Offensive Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  94.24860853432281
Confusion Matrix: 
 [[508   0]
 [ 31   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       508
           1       0.00      0.00      0.00        31

    accuracy                           0.94       539
   macro avg       0.47      0.50      0.49       539
weighted avg       0.89      0.94      0.91       539

-----------------------------------------------------------

Offensive Accuracy score(test data with multiple features): 94.24860853432281
Confusion Matrix: 
 [[508   0]
 [ 31   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       508
           1       0.00      0.00      0.00        31

    accuracy                           0.94       539
   macro avg       0.47      0.50      0.49       539
weighted avg       0.89      0.94      0

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

In [304]:
#Naive bays
import numpy
from sklearn.naive_bayes import MultinomialNB
#P=threshold*(content)+(1-threshold)*(context)
#ensemble model of comments feature and context feature
abusive_type = ["Rude","Figurative","Offensive","Dirty"]
report=[]
for threshold in numpy.arange(0.1,1.1,0.1):
    for i in range(0,4):
        Naive_bayes_1 = MultinomialNB()
        Naive_bayes_2 = MultinomialNB()

        Naive_bayes_1.fit(Train_X_Tfidf, Train_Y[abusive_type[i]])
        Naive_bayes_2.fit(train_feat_context, Train_Y[abusive_type[i]])

        predictions_NB_Train1 = Naive_bayes_1.predict_proba(Train_X_Tfidf)
        predictions_NB_Train2 = Naive_bayes_2.predict_proba(train_feat_context)

        predictions_NB_Test1 = Naive_bayes_1.predict_proba(Test_X_Tfidf)
        predictions_NB_Test2 = Naive_bayes_2.predict_proba(test_feat_context)

        predictions_NB_Train = (predictions_NB_Train1*threshold)+(predictions_NB_Train2*(1-threshold))
        predictions_NB_Test = (predictions_NB_Test1*threshold)+(predictions_NB_Test2*(1-threshold))

        predictions_class_Train = predict_class(predictions_NB_Train,threshold)
        predictions_class_Test = predict_class(predictions_NB_Test,threshold)

        report_performance_threshold_1_minus_threshold(predictions_class_Train, predictions_class_Test, abusive_type[i],threshold)

Rude Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  64.19294990723562
Confusion Matrix: 
 [[ 25 178]
 [ 15 321]]
PRF: 
               precision    recall  f1-score   support

           0       0.62      0.12      0.21       203
           1       0.64      0.96      0.77       336

    accuracy                           0.64       539
   macro avg       0.63      0.54      0.49       539
weighted avg       0.64      0.64      0.56       539

-----------------------------------------------------------

Rude Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  60.278422273781906
Confusion Matrix: 
 [[  72  793]
 [  63 1227]]
PRF: 
               precision    recall  f1-score   support

           0       0.53      0.08      0.14       865
           1       0.61      0.95      0.74      1290

    accuracy                           0.60      2155
   macro avg       0.57      0.52      0.44      2155


PRF: 
               precision    recall  f1-score   support

           0       0.56      0.13      0.21       865
           1       0.61      0.93      0.74      1290

    accuracy                           0.61      2155
   macro avg       0.59      0.53      0.47      2155
weighted avg       0.59      0.61      0.53      2155

___________________________________________________________________________________________________________

Figurative Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  84.97217068645641
Confusion Matrix: 
 [[456  52]
 [ 29   2]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      0.90      0.92       508
           1       0.04      0.06      0.05        31

    accuracy                           0.85       539
   macro avg       0.49      0.48      0.48       539
weighted avg       0.89      0.85      0.87       539

---------------------------------------------------------

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       508
           1       0.00      0.00      0.00        31

    accuracy                           0.94       539
   macro avg       0.47      0.50      0.49       539
weighted avg       0.89      0.94      0.91       539

-----------------------------------------------------------

Figurative Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  95.22041763341068
Confusion Matrix: 
 [[2052    0]
 [ 103    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      2052
           1       0.00      0.00      0.00       103

    accuracy                           0.95      2155
   macro avg       0.48      0.50      0.49      2155
weighted avg       0.91      0.95      0.93      2155

____________________________________________________________________________________________________

Offensive Accuracy score(test data with ensemble (1 model: content feature, 1 model: context feature):  94.24860853432281
Confusion Matrix: 
 [[508   0]
 [ 31   0]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       508
           1       0.00      0.00      0.00        31

    accuracy                           0.94       539
   macro avg       0.47      0.50      0.49       539
weighted avg       0.89      0.94      0.91       539

-----------------------------------------------------------

Offensive Accuracy score(train data with ensemble (1 model: content feature, 1 model: context feature):  93.7354988399072
Confusion Matrix: 
 [[2020    0]
 [ 135    0]]
PRF: 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      2020
           1       0.00      0.00      0.00       135

    accuracy                           0.94      2155
   macro avg       0.47      0.50      0.48   

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 

In [182]:
a = Train_X.to_frame(name='comment')
a['Rude'] = predictions_NB_Rude_Train[:,1]
a['Figurative'] = predictions_NB_Figurative_Train[:,1]
a['Offensive'] = predictions_NB_Offensive_Train[:,1]
a['Dirty'] = predictions_NB_Dirty_Train[:,1]
a.to_csv('nb-tfidf-predict_train.csv')

b = Test_X.to_frame(name='comment')
b['Rude'] = predictions_NB_Rude_Test[:,1]
b['Figurative'] = predictions_NB_Figurative_Test[:,1]
b['Offensive'] = predictions_NB_Offensive_Test[:,1]
b['Dirty'] = predictions_NB_Dirty_Test[:,1]
b.to_csv('nb-tfidf-predict_test.csv')

c = Train_X.to_frame(name='comment')
c['Rude'] = Train_Y['Rude']
c['Figurative'] = Train_Y['Figurative']
c['Offensive'] = Train_Y['Offensive']
c['Dirty'] = Train_Y['Dirty']
c.to_csv('train_and_label.csv')

d = Test_X.to_frame(name='comment')
d['Rude'] = Test_Y['Rude']
d['Figurative'] = Test_Y['Figurative']
d['Offensive'] = Test_Y['Offensive']
d['Dirty'] = Test_Y['Dirty']
d.to_csv('test_and_label.csv')

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [2]:
import pandas as pd
Corpus = pd.read_excel('corpus.xlsx')
Corpus

,comments,comment_id,reaction1,reaction2,reaction3,reaction_num,reply_num,comment_time,type1,type2,Rude,Figurative,Offensive,Dirty
0,อย่างที่กูบอกเป๊ะ แม่งยื้อ ออกข่าวว่ากำลังรับผ...,10157700000000000,like,sad,angry,148,0,43681.89306,1,NaN,1,0,0,0
1,แพรวามึงเกิดมาทำเหี้ยอะไรวะกูถามจริง เกิดมาเป็...,10157700000000000,NaN,NaN,NaN,42,2,43681.89028,1,4.0,1,1,1,0
2,รอเรื่องเงียบ มีข่าวอื่นมากลบ ก็ดื้อต่อไปไม่จ่...,10157700000000000,NaN,NaN,NaN,8,0,43681.89306,1,NaN,0,0,0,0
3,ให้กรมที่ดินมาประเมินกุว่าไม่ถึง50 ล้าน ตั้งรา...,10157700000000000,NaN,NaN,NaN,10,1,43681.89306,1,NaN,1,0,0,0
4,คนบ้าๆไร้ค่าคนเดียวทำอีกหลายครอบครัวเค้าเดือดร...,10157700000000000,NaN,NaN,NaN,3,0,43681.89514,1,NaN,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,สวยๆแบบกุยังไม่กล้าหลอกใครเลย มีโดนเขาหลอก แถม...,10157700000000000,NaN,NaN,NaN,1,0,43670.91528,1,NaN,1,0,0,0
458,ไม่น่ารับแจ้งความเคสแบบนี้ มึงจะมีเงินเยอะแจกค...,10157700000000000,NaN,NaN,NaN,1,0,43671.41319,1,NaN,1,0,0,0
459,What 2 ล้านขอกูได้เป็นเมียเป็นตัวเป็นตนได้เลยน...,10157700000000000,NaN,NaN,NaN,4,5,43670.91736,1,NaN,1,0,0,0
460,ไม่ได้แดกผมหรอกคัฟ เพราะผมก็ไม่มีจะแดก ๕๕,10157700000000000,NaN,NaN,NaN,3,0,43670.91597,1,NaN,1,0,0,0
